In [1]:
import base64
import collections
import datetime
import json
import os
import random
import time

import scipy
from scipy import ndimage
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from pylab import *
from matplotlib.dates import date2num , DateFormatter
from PIL import Image

np.random.seed(1337)

%matplotlib inline
sns.set(font_scale=1.0)
rcParams['figure.figsize'] = 8, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# Load the Data

In [2]:
d = pd.read_json('tmp/raw_data.json')
d.index.name = 'id'
d = d[['price', 'bedrooms']]
print len(d)

124011


In [3]:
d.dtypes

price       int64
bedrooms    int64
dtype: object

In [4]:
d.price.head()

id
0       2950
1       2850
10      3000
100     3758
1000    3300
Name: price, dtype: int64

# Price Bucket with Same Number of Bedrooms

In [5]:
d['price_br_q'] = np.nan
for b in d.bedrooms.value_counts().index:
    d.loc[d.bedrooms == b, 'price_br_q'] = \
        d[d.bedrooms == b].price.rank() / len(d[d.bedrooms == b])

# Apts with Approx. Same Price

In [8]:
d['rounded_price'] = (d.price / 100).astype(int64)
cnt = collections.Counter(d.rounded_price.tolist())
d['num_apts_with_approx_same_price'] = d.rounded_price.map(cnt).astype(float64)

# Save the Data

In [9]:
del d['bedrooms']
del d['rounded_price']

In [11]:
d.price = d.price.astype(float64)

In [12]:
d.to_csv('tmp/features_price.csv')